<a href="https://colab.research.google.com/github/JacquesYL/MEF/blob/main/Efficient_frontier_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import key librares and modules
import pandas as pd
import numpy as np

# Import datetime module that comes pre-installed in Python
# datetime offers classes that work with date & time information
import datetime as dta

import matplotlib.pyplot as plt

!pip install seaborn
import seaborn as sns

import plotly.express as px

In [ ]:
# Define a function that performs interactive data visualization using Plotly Express
def plot_financial_data(df, title):

    fig = px.line(title = title)

    # For loop that plots all stock prices in the pandas dataframe df
    # Note that index starts with 1 because we want to skip the date column

    #skip the date, so the column starts with index = 1 (date is when index = 0)
    for i in df.columns[1:]:
        fig.add_scatter(x = df['Date'], y = df[i], name = i)
        fig.update_traces(line_width = 1)
        fig.update_layout({'plot_bgcolor': "white"})

    fig.show()

In [ ]:
# Function to scale stock prices based on their initial starting price
# The objective of this function is to set all prices to start at a value of 1
def price_scaling(raw_prices_df):
    scaled_prices_df = raw_prices_df.copy()
    for i in raw_prices_df.columns[1:]:
          scaled_prices_df[i] = raw_prices_df[i]/raw_prices_df[i][0]
    return scaled_prices_df

In [ ]:
import random

def generate_portfolio_weights(n):
    weights = []
    for i in range(n):
        weights.append(random.random())

    # let's make the sum of all weights add up to 1
    weights = weights/np.sum(weights)
    return weights

In [ ]:
# Assume that we have $1,000,000 that we would like to invest in one or more of the selected stocks
# Let's create a function that receives the following arguments:
      # (1) Stocks closing prices
      # (2) Random weights
      # (3) Initial investment amount
# The function will return a DataFrame that contains the following:
      # (1) Daily value (position) of each individual stock over the specified time period
      # (2) Total daily value of the portfolio
      # (3) Percentage daily return

def asset_allocation(df, weights, initial_investment):
    portfolio_df = df.copy()

    # Scale stock prices using the "price_scaling" function that we defined earlier (Make them all start at 1)
    scaled_df = price_scaling(df)

    for i, stock in enumerate(scaled_df.columns[1:]):
        portfolio_df[stock] = scaled_df[stock] * weights[i] * initial_investment

    # Sum up all values and place the result in a new column titled "portfolio value [$]"
    # Note that we excluded the date column from this calculation
    portfolio_df['Portfolio Value [$]'] = portfolio_df[portfolio_df != 'Date'].sum(axis = 1, numeric_only = True)

    # Calculate the portfolio percentage daily return and replace NaNs with zeros
    portfolio_df['Portfolio Daily Return [%]'] = portfolio_df['Portfolio Value [$]'].pct_change(1) * 100
    portfolio_df.replace(np.nan, 0, inplace = True)

    return portfolio_df

In [ ]:
# Let's define the simulation engine function
# The function receives:
    # (1) portfolio weights
    # (2) initial investment amount
# The function performs asset allocation and calculates portfolio statistical metrics including Sharpe ratio
# The function returns:
    # (1) Expected portfolio return
    # (2) Expected volatility
    # (3) Sharpe ratio
    # (4) Return on investment
    # (5) Final portfolio value in dollars


def simulation_engine(weights, initial_investment):
    # Perform asset allocation using the random weights (sent as arguments to the function)
    portfolio_df = asset_allocation(close_price_df, weights, initial_investment)

    # Calculate the return on the investment
    # Return on investment is calculated using the last final value of the portfolio compared to its initial value
    return_on_investment = ((portfolio_df['Portfolio Value [$]'][-1:] - portfolio_df['Portfolio Value [$]'][0])/
                             portfolio_df['Portfolio Value [$]'][0]) * 100

    # Daily change of every stock in the portfolio (Note that we dropped the date, portfolio daily worth and daily % returns)
    portfolio_daily_return_df = portfolio_df.drop(columns = ['Date', 'Portfolio Value [$]', 'Portfolio Daily Return [%]'])
    portfolio_daily_return_df = portfolio_daily_return_df.pct_change(1)

    # Portfolio Expected Return formula
    expected_portfolio_return = np.sum(weights * portfolio_daily_return_df.mean() ) * 252

    # Portfolio volatility (risk) formula
    # The risk of an asset is measured using the standard deviation which indicates the dispertion away from the mean
    # The risk of a portfolio is not a simple sum of the risks of the individual assets within the portfolio
    # Portfolio risk must consider correlations between assets within the portfolio which is indicated by the covariance
    # The covariance determines the relationship between the movements of two random variables
    # When two stocks move together, they have a positive covariance when they move inversely, the have a negative covariance

    covariance = portfolio_daily_return_df.cov() * 252
    expected_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))

    # Check out the chart for the 10-years U.S. treasury at https://ycharts.com/indicators/10_year_treasury_rate
    rf = 0.0425 # Try to set the risk free rate of return to 1% (assumption)

    # Calculate Sharpe ratio
    sharpe_ratio = (expected_portfolio_return - rf)/expected_volatility
    return expected_portfolio_return, expected_volatility, sharpe_ratio, portfolio_df['Portfolio Value [$]'][-1:].values[0], return_on_investment.values[0]


In [ ]:
# Let's read multiple stocks data contained in "stock_prices.csv" attached in the course package
# The Appendix includes details on how to obtain this data using yfinance and Pandas Datareader
# Note that yfinance and Pandas Datareader might experience outage in some geographical regions

# We will focus our analysis on U.S. stocks, similar analysis could be performed on Asian, European or African stocks
# AMZN: Amazon Inc. - Multinational tech company focusing on e-commerce, cloud computing, and artificial intelligence
# JPM: JPMorgan Chase and Co. - Multinational investment bank and financial services holding company
# META: Meta Platforms, formerly named Facebook Inc. - META owns Facebook, Instagram, and WhatsApp
# PG: Procter and Gamble (P&G) - Multinational consumer goods corporation
# GOOG: Google (Alphabet Inc.) - Multinational company that focuses on search engine tech, e-commerce, Cloud and AI
# CAT: Caterpillar - World's largest construction-equipment manufacturer
# PFE: Pfizer Inc. - Multinational pharmaceutical and biotechnology corporation
# EXC: Exelon - An American Fortune 100 energy company
# DE: Deere & Company (John Deere) - Manufactures agricultural machinery and heavy equipment
# JNJ: Johnson & Johnson - A multinational corporation that develops medical devices and pharmaceuticals

close_price_df = pd.read_csv('stock_prices.csv')
close_price_df

,Date,AMZN,CAT,DE,EXC,GOOGL,JNJ,JPM,META,PFE,PG
0,1/2/2014,19.898500,69.512543,75.620926,14.121004,27.855856,71.443314,45.640057,54.709999,20.879053,61.941517
1,1/3/2014,19.822001,69.473892,75.956062,13.835152,27.652653,72.086861,45.992889,54.560001,20.920183,61.872295
2,1/6/2014,19.681499,68.561180,75.327690,13.923512,27.960960,72.463608,46.259468,57.200001,20.940746,62.018425
3,1/7/2014,19.901501,68.785461,75.662811,13.996271,28.500000,74.001869,45.726311,57.919998,21.070980,62.618301
4,1/8/2014,20.096001,68.947906,74.850121,13.965082,28.559309,73.899841,46.157543,58.230000,21.214928,61.710812
...,...,...,...,...,...,...,...,...,...,...,...
2252,12/12/2022,90.550003,233.059998,437.049988,42.500000,93.309998,177.839996,134.210007,114.709999,52.160000,152.470001
2253,12/13/2022,92.489998,235.490005,437.190002,42.540001,95.629997,179.210007,134.080002,120.150002,53.070000,152.240005
2254,12/14/2022,91.580002,234.479996,438.440002,42.820000,95.070000,179.759995,133.410004,121.589996,54.480000,152.839996
2255,12/15/2022,88.449997,230.660004,429.790008,42.380001,90.860001,177.490005,130.100006,116.150002,53.610001,151.110001


In [ ]:
# The objective of this code cell is to calculate the percentage daily return
# We will perform this calculation on all stocks except for the first column which is "Date"
daily_returns_df = close_price_df.iloc[:, 1:].pct_change() * 100
daily_returns_df.replace(np.nan, 0, inplace = True)
daily_returns_df

,AMZN,CAT,DE,EXC,GOOGL,JNJ,JPM,META,PFE,PG
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.384451,-0.055602,0.443179,-2.024307,-0.729481,0.900780,0.773077,-0.274169,0.196992,-0.111753
2,-0.708814,-1.313748,-0.827284,0.638662,1.114930,0.522629,0.579608,4.838708,0.098293,0.236179
3,1.117807,0.327126,0.444884,0.522567,1.927829,2.122805,-1.152537,1.258737,0.621916,0.967255
4,0.977313,0.236162,-1.074095,-0.222838,0.208102,-0.137872,0.943073,0.535223,0.683156,-1.449240
...,...,...,...,...,...,...,...,...,...,...
2252,1.638800,2.538609,0.515165,2.607441,0.517070,1.194942,1.551152,-1.026749,0.850732,1.027036
2253,2.142457,1.042653,0.032036,0.094120,2.486336,0.770361,-0.096867,4.742396,1.744632,-0.150847
2254,-0.983886,-0.428897,0.285917,0.658201,-0.585588,0.306896,-0.499700,1.198498,2.656868,0.394108
2255,-3.417782,-1.629133,-1.972903,-1.027554,-4.428315,-1.262789,-2.481072,-4.474048,-1.596914,-1.131900


In [ ]:
# Let's test out the "simulation_engine" function and print out statistical metrics
# Define the initial investment amount
weights = np.ones(10)/10
initial_investment = 1000000
portfolio_metrics = simulation_engine(weights, initial_investment)

print('Expected Portfolio Annual Return = {:.2f}%'.format(portfolio_metrics[0] * 100))
print('Portfolio Standard Deviation (Volatility) = {:.2f}%'.format(portfolio_metrics[1] * 100))
print('Sharpe Ratio = {:.2f}'.format(portfolio_metrics[2]))
print('Portfolio Final Value = ${:.2f}'.format(portfolio_metrics[3]))
print('Return on Investment = {:.2f}%'.format(portfolio_metrics[4]))

Expected Portfolio Annual Return = 16.30%
Portfolio Standard Deviation (Volatility) = 18.02%
Sharpe Ratio = 0.74
Portfolio Final Value = $3203874.76
Return on Investment = 220.39%


In [ ]:
# Let's test out the "simulation_engine" function and print out statistical metrics
# Define the initial investment amount
initial_investment = 1000000
portfolio_metrics = simulation_engine(weights, initial_investment)

In [ ]:
# Set the number of simulation runs
sim_runs = 10000
initial_investment = 1000000

# Placeholder to store all weights
n = 10
weights_runs = np.zeros((sim_runs, n))

# Placeholder to store all Sharpe ratios
sharpe_ratio_runs = np.zeros(sim_runs)

# Placeholder to store all expected returns
expected_portfolio_returns_runs = np.zeros(sim_runs)

# Placeholder to store all volatility values
volatility_runs = np.zeros(sim_runs)

# Placeholder to store all returns on investment
return_on_investment_runs = np.zeros(sim_runs)

# Placeholder to store all final portfolio values
final_value_runs = np.zeros(sim_runs)

for i in range(sim_runs):
    # Generate random weights
    weights = generate_portfolio_weights(n)
    # Store the weights
    weights_runs[i,:] = weights

    # Call "simulation_engine" function and store Sharpe ratio, return and volatility
    # Note that asset allocation is performed using the "asset_allocation" function
    expected_portfolio_returns_runs[i], volatility_runs[i], sharpe_ratio_runs[i], final_value_runs[i], return_on_investment_runs[i] = simulation_engine(weights, initial_investment)
    print("Simulation Run = {}".format(i))
    print("Weights = {}, Final Value = ${:.2f}, Sharpe Ratio = {:.2f}".format(weights_runs[i].round(3), final_value_runs[i], sharpe_ratio_runs[i]))
    print('\n')


Streaming output truncated to the last 5000 lines.
Simulation Run = 8750
Weights = [0.005 0.181 0.065 0.084 0.026 0.057 0.068 0.129 0.192 0.193], Final Value = $2886735.13, Sharpe Ratio = 0.62


Simulation Run = 8751
Weights = [0.183 0.086 0.105 0.139 0.063 0.017 0.159 0.033 0.122 0.094], Final Value = $3400365.44, Sharpe Ratio = 0.69


Simulation Run = 8752
Weights = [0.15  0.084 0.028 0.063 0.066 0.103 0.119 0.196 0.057 0.135], Final Value = $2986039.19, Sharpe Ratio = 0.61


Simulation Run = 8753
Weights = [0.046 0.143 0.126 0.109 0.059 0.094 0.081 0.157 0.085 0.1  ], Final Value = $3170364.93, Sharpe Ratio = 0.65


Simulation Run = 8754
Weights = [0.142 0.092 0.116 0.074 0.007 0.087 0.059 0.146 0.139 0.137], Final Value = $3216896.06, Sharpe Ratio = 0.67


Simulation Run = 8755
Weights = [0.14  0.043 0.028 0.146 0.23  0.089 0.037 0.15  0.093 0.044], Final Value = $3088209.19, Sharpe Ratio = 0.61


Simulation Run = 8756
Weights = [0.091 0.107 0.023 0.165 0.136 0.146 0.028 0.074 0.13

In [ ]:
# List all Sharpe ratios generated from the simulation
sharpe_ratio_runs

array([0.67693584, 0.74368861, 0.71047646, ..., 0.71036324, 0.7352952 ,
       0.7127702 ])

In [ ]:
# Return the index of the maximum Sharpe ratio (Best simulation run)
print(sharpe_ratio_runs.argmax())

318


In [ ]:
# Return the maximum Sharpe ratio value
print(sharpe_ratio_runs.max().round(4))

0.8214


In [ ]:
# Obtain the portfolio weights that correspond to the maximum Sharpe ratio (Golden set of weights!)
print(weights_runs[sharpe_ratio_runs.argmax(), :])

[0.19370369 0.02302675 0.25033911 0.23385147 0.03638116 0.08407225
 0.01549497 0.02736974 0.03471493 0.10104592]


In [ ]:
# Return Sharpe ratio, volatility corresponding to the best weights allocation (maximum Sharpe ratio)
optimal_portfolio_return, optimal_volatility, optimal_sharpe_ratio, highest_final_value, optimal_return_on_investment = simulation_engine(weights_runs[sharpe_ratio_runs.argmax(), :], initial_investment)

print('Best Portfolio Metrics Based on {} Monte Carlo Simulation Runs:'.format(sim_runs))
print('  - Portfolio Expected Annual Return = {:.02f}%'.format(optimal_portfolio_return * 100))
print('  - Portfolio Standard Deviation (Volatility) = {:.02f}%'.format(optimal_volatility * 100))
print('  - Sharpe Ratio = {:.02f}'.format(optimal_sharpe_ratio))
print('  - Final Value = ${:.02f}'.format(highest_final_value))
print('  - Return on Investment = {:.02f}%'.format(optimal_return_on_investment))

Best Portfolio Metrics Based on 10000 Monte Carlo Simulation Runs:
  - Portfolio Expected Annual Return = 17.64%
  - Portfolio Standard Deviation (Volatility) = 17.82%
  - Sharpe Ratio = 0.75
  - Final Value = $3748355.75
  - Return on Investment = 274.84%


In [ ]:
# Return Sharpe ratio, volatility corresponding to the GMV
gmv_portfolio_return, gmv_volatility, gmv_sharpe_ratio, gmv_final_value, gmv_return_on_investment = simulation_engine(weights_runs[volatility_runs.argmin(), :], initial_investment)

print('GMV Portfolio Metrics Based on {} Monte Carlo Simulation Runs:'.format(sim_runs))
print('  - Portfolio Expected Annual Return = {:.02f}%'.format(gmv_portfolio_return * 100))
print('  - Portfolio Standard Deviation (Volatility) = {:.02f}%'.format(gmv_volatility * 100))
print('  - Sharpe Ratio = {:.02f}'.format(gmv_sharpe_ratio))
print('  - Final Value = ${:.02f}'.format(gmv_final_value))
print('  - Return on Investment = {:.02f}%'.format(gmv_return_on_investment))


GMV Portfolio Metrics Based on 10000 Monte Carlo Simulation Runs:
  - Portfolio Expected Annual Return = 13.86%
  - Portfolio Standard Deviation (Volatility) = 15.90%
  - Sharpe Ratio = 0.60
  - Final Value = $2835879.13
  - Return on Investment = 183.59%


In [ ]:
# Create a DataFrame that contains volatility, return, and Sharpe ratio for all simualation runs
sim_out_df = pd.DataFrame({'Volatility': volatility_runs.tolist(), 'Portfolio_Return': expected_portfolio_returns_runs.tolist(), 'Sharpe_Ratio': sharpe_ratio_runs.tolist() })
sim_out_df

,Volatility,Portfolio_Return,Sharpe_Ratio
0,0.180116,0.169025,0.702460
1,0.183595,0.154671,0.610970
2,0.176215,0.152872,0.626350
3,0.192294,0.170841,0.667417
4,0.179131,0.155974,0.633466
...,...,...,...
9995,0.213562,0.182463,0.655378
9996,0.192387,0.172158,0.673942
9997,0.187221,0.169085,0.676128
9998,0.173040,0.147115,0.604571


In [ ]:
# Let's highlight the point with the highest Sharpe ratio
import plotly.graph_objects as go

factor = 1.2

# Calculate the extended x-coordinate
extended_volatility = optimal_volatility * factor

# Calculate the slope of the line between (0, 0.03) and (optimal_volatility, optimal_portfolio_return)
slope = (optimal_portfolio_return - 0.03) / optimal_volatility

# Compute the extended y-coordinate using the line equation: y = slope * x + intercept
extended_return = slope * extended_volatility + 0.03

fig = px.scatter(sim_out_df,
                 x='Volatility',
                 y='Portfolio_Return',
                 color='Sharpe_Ratio',
                 size='Sharpe_Ratio',
                 size_max=8,  # Adjust this value as needed
                 hover_data=['Sharpe_Ratio'])
# GMV portfolio
fig.add_trace(go.Scatter(x = [gmv_volatility], y = [gmv_portfolio_return],
                         mode = 'markers',
                         name = 'GMV',
                         marker=dict(
                         size=10,
                         color='green',
                         symbol='diamond'  # change to desired marker type
)
))
fig.add_trace(go.Scatter(x=[optimal_volatility],
                         y=[optimal_portfolio_return],
                         mode='markers',
                         name='Optimal Risky Portfolio',
                         marker=dict(
                         size=15,
                         color='red',
                         symbol='circle'  # change to desired marker type
)
))
fig.add_trace(go.Scatter(x = [0], y = [0.03],
                         mode = 'markers',
                         name = 'Risk-free asset',
                         marker=dict(
                         size=10,
                         color='blue',
                         symbol='square'  # change to desired marker type
)
))

# Add the extended line trace
fig.add_trace(go.Scatter(
    x=[0, extended_volatility],
    y=[0.03, extended_return],
    mode='lines',
    name='Capital Market Line (CML)',
    line=dict(color='black', dash='dash')
))
fig.update_layout(coloraxis_colorbar=dict(y=0.7, dtick=5))
fig.update_layout({'plot_bgcolor': "white"})

fig.update_layout(
    coloraxis_colorbar=dict(
        y=0.7,          # vertical position
        x=0.95,         # horizontal position, adjust as needed (0 is left, 1 is right)
        xanchor='left', # aligns the left side of the colorbar to the x position
        yanchor='middle',
        dtick=5
    )
)

fig.update_layout(coloraxis_showscale=False)

fig.show()

